In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import MWETokenizer
import multiprocessing
import pandas as pd
import re
import sqlite3

cores = multiprocessing.cpu_count()

In [5]:
spltr = re.compile( r'\W+' )
#tag_map = []
arXiv_start = 0

def get_abstracts_indices_from_DBLP():
    papers = pd.read_csv( '../../data-kw.csv' )
    # TODO: Get from whole dataset (too)
    for i, abstract in enumerate( papers['ABSTRACT'] ):
        index = papers['INDEX'][i]  # NOTE: This is a string too, yeah??
        if isinstance( abstract, float ):
            #print( 'NO ABSTRACT FOUND at Index = '+ index )
            continue
        ab_words = [ w for w in spltr.split( abstract.lower() ) if w != '' ]
        #tag_map.append( index )
        yield TaggedDocument( words=ab_words, tags=[int(index)] )  # NOTE: WAS tags=[i]
        # TODO: Consider having the list of references as multiple tags/labels?
        
def get_abstracts_indices_from_arXiv():
    arXiv_start = len( tag_map )
    conn = sqlite3.connect( '../../arxiv-server/paper_trail.db' )
    curs = conn.cursor()
    for i, row in enumerate( curs.execute( 'SELECT id, abstract FROM abstracts' ) ):
        index = row[0]   # NOTE: These are strings, yeah??
        abstract = row[1]
        ab_words = [ w for w in spltr.split( abstract.lower() ) if w != '' ]
        #tag_map.append( index )
        yield TaggedDocument( words=ab_words, tags=[int(index)])  # NOTE: WAS tags=[arXiv_start + i]
    conn.close()

In [16]:
def transform_DBLP_data():
    papers = pd.read_csv( '../../data-kw.csv' )
    with open( '../../DBLP_ids_abstracts.txt', 'w' ) as f:
        for i, abstract in enumerate( papers['ABSTRACT'] ):
            index = papers['INDEX'][i]
            f.write( str(index) + '|' + str(abstract) + '\n' )
            
transform_DBLP_data()

In [7]:
DBLP_docs = list( get_abstracts_indices_from_DBLP() )
#arXiv_docs = list( get_abstracts_indices_from_arXiv() )
arXiv_docs = []
docs = DBLP_docs + arXiv_docs
print( len(DBLP_docs) )
print( len(arXiv_docs) )
print( len(docs) )

265964
0
265964


In [22]:
def get_paper_abstract( idx ):
    """ Given a model doc tag, output the respective DBLP or arXiv paper abstract"""
    return ' '.join( docs[idx].words )

In [10]:
model = Doc2Vec( docs, size=200, window=8, min_count=2, workers=cores )

In [11]:
model.save( 'doc2vecmodel_s200w8mc2' )

In [12]:
model2 = Doc2Vec.load( 'doc2vecmodel_s400w8mc2' )

In [ ]:
from random import shuffle
import datetime

# From gensim doc:
simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=400, window=5, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=400, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=400, window=10, negative=5, hs=0, min_count=2, workers=cores),
    # My others:
    Doc2Vec(size=200, workers=cores),
    Doc2Vec(size=300, workers=cores),
    Doc2Vec(size=400, workers=cores)
]

simple_models[0].build_vocab(alldocs)  # PV-DM/concat requires one special NULL word so it serves as template
for model in simple_models[1:]:
    model.reset_from(simple_models[0])
    
alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes

for epoch in range(passes):
    

In [24]:
model.docvecs.most_similar( 397 )

[(363, 0.961861789226532),
 (92, 0.9612609148025513),
 (277, 0.956291913986206),
 (441, 0.9476752877235413),
 (256, 0.9442665576934814),
 (186, 0.9399662613868713),
 (264, 0.933863639831543),
 (368, 0.9276899695396423),
 (176, 0.9242762923240662),
 (291, 0.9103182554244995)]

In [25]:
test_idx = 397
compare = 3

compare = min( compare, 10 )
print( 'TEST ABSTRACT:\n' + get_paper_abstract(test_idx) )
print( '\n  SIMILAR TO:\n' )
for idx_score in model.docvecs.most_similar( test_idx )[:compare]:
    print( get_paper_abstract(idx_score[0]) )
    print( '  (id = '+ str(idx_score[0]) + ', with a score of '+ str(idx_score[1]) +')\n' )

TEST ABSTRACT:
in this paper we present our research on social interaction in co located handheld augmented reality ar games these games are characterized by shared physical spaces that promote physical awareness among players and individual gaming devices that support both public and private information one result of our exploration of the design and evaluation of such games is a prototype called bragfish through bragfish we aim to investigate the connections between the observed game experience focusing on social and physical interaction and the designed affordances of our ar handheld game our evaluation of bragfish shows that most of our participants form strategies for social play by leveraging visual aural and physical cues from the shared space moreover we use this as an example to motivate discussions on how to improve social play experiences for co located handheld games by designing for shared spaces

  SIMILAR TO:

avatars are traditionally understood as representing their hu

In [6]:
def check_recommendations( recs, acts ):
    """ recs = list of doc IDs as the recommended references/citations
        acts = list of doc IDs as the actual references/citations
        NOTE: the indices are given as the 'docs' indices, which then refer to actual doc IDs
    """
    matches = 0
    nomatches = 0
    for rec in recs:
        if rec in acts:
            matches += 1
        else:
            nomatches += 1
    return matches, nomatches

In [7]:
papers = pd.read_csv( '../../data-kw.csv' )

In [8]:
def get_references( index ):
    """ Given a 'docs' index, get the respective document's references
    """
    if index >= arXiv_start:
        print( "DEBUG: WE DON'T HAVE REFERENCES FOR THE ARXIV PAPERS" )
        return []
    refs = list( papers[papers['INDEX'] == tag_map[index]]['REF_ID'] )
    if ';' in refs[0]:
        refs = refs[0].split(';')
    refs = list( map(int, refs) )
    return refs

In [11]:
def get_recommendations( index ):
    # TODO: Update this with infer_vector stuff
    #       and better logic in general.
    #doc_ids = [ tag_map[index] for index, _ in model.docvecs.most_similar( index) ]
    
    most_sim = 10
    vec = model.infer_vector( get_paper_abstract(index).split() )
    # Code you'll find on nycdatascience.com:
    cossims = list( map( lambda v: cossim(vec, v), model.docvecs ) )
    sim_ids = argmaxn( cossims, most_sim )
    #for i in range(most_sim):
        #print( sim_ids[i], cossims[sim_ids[i]] )
    doc_ids = [ tag_map[i] for i in sim_ids ]
    return doc_ids

In [18]:
tot_matches = 0
tot_nomatches = 0
for test in range(1000):
    recs = get_recommendations( test )
    refs = get_references( test )
    matches, nomatches = check_recommendations( recs, refs )
    tot_matches += matches
    tot_nomatches += nomatches
print( str(tot_matches) + " " + str(tot_nomatches) )

3 9997


In [10]:
import numpy as np
# Code you'll find on nycdatascience.com:
def cossim( v1, v2 ):
    return np.dot(v1, v2) / np.sqrt( np.dot(v1, v1) ) / np.sqrt( np.dot(v2, v2) )
def argmaxn( l, n ):
    l_copy = list(l)
    args = []
    for i in range(n):
        arg = np.argmax(l_copy)
        args.append(arg)
        l_copy[arg] = -float('inf')
    return args

In [6]:
# Code you'll find on that other website:
import lda
from sklearn.feature_extraction.text import CountVectorizer

# TODO: max_features was initially 10000
cvectorizer = CountVectorizer( min_df=4, max_features=5000, stop_words='english' )
abstracts = [ ' '.join( x.words ) for x in docs ]
cvz = cvectorizer.fit_transform( abstracts )

In [7]:
n_topics = 20
n_iter = 2000
lda_model = lda.LDA( n_topics=n_topics, n_iter=n_iter )
print( "DONE LDA" )
X_topics = lda_model.fit_transform( cvz )

DONE LDA


In [8]:
n_top_words = 8
topic_summaries = []

topic_word = lda_model.topic_word_
vocab = cvectorizer.get_feature_names()
for i, topic_dist in enumerate( topic_word ):
    topic_words = np.array( vocab )[ np.argsort(topic_dist) ][ : -(n_top_words+1):-1 ]
    topic_summaries.append( ' '.join( topic_words ) )
    print( 'Topic {}: {}'.format(i, ' '.join( topic_words) ) )

Topic 0: user users interaction paper human interface virtual information
Topic 1: image method images based using paper object proposed
Topic 2: graph problem algorithm time graphs number log set
Topic 3: algorithm problem algorithms problems optimization search solution based
Topic 4: network networks nodes routing wireless traffic sensor performance
Topic 5: data query database queries mining databases paper xml
Topic 6: model software design language systems paper based approach
Topic 7: security protocol key scheme based protocols attacks paper
Topic 8: functions codes quantum function linear complexity number finite
Topic 9: logic language paper semantics systems model theory based
Topic 10: model network networks models social time analysis structure
Topic 11: learning data classification based method methods clustering proposed
Topic 12: method model distribution matrix estimation algorithm linear probability
Topic 13: service services web systems applications distributed based

In [48]:
model.most_similar( positive=['classification'], negative=['recognition'] )

[('clustering', 0.3813590407371521),
 ('ranking', 0.37842971086502075),
 ('ensemble', 0.3661080598831177),
 ('fusion', 0.3224477767944336),
 ('classifiers', 0.32240602374076843),
 ('inference', 0.30264925956726074),
 ('clusterings', 0.2996658682823181),
 ('pruning', 0.2980048656463623),
 ('boosting', 0.2952820360660553),
 ('encoding', 0.293839693069458)]

In [10]:
model.n_similarity( ['human', 'chess'], ['deep', 'blue'] )

0.15382886432060081

In [ ]:
# When checking user's input, (if necessary) see if the keywords are present in the model or not